# Migrating CSV Data into a Relational Database
# Step 1: Import CSV Data into Pandas Dataframes

In [1]:
import pandas as pd

# Reading the csv file

In [2]:
df = pd.read_csv('cleaned_data.csv')
print(f"The dataset contains {len(df)} Condo Rent listings")
pd.set_option('display.max_columns', len(df.columns)) # To view all columns
pd.set_option('display.max_rows', 100)
df.head(10)

The dataset contains 2554 Condo Rent listings


,id,id_person,name,surname,zip,city,country,email,birth_date,id_account,account_type,id_transaction,transaction_type,transaction_date,transaction_amount
0,0,13,Aaya,Añon,10013,Ajman,United Arab Emirates,aaya.añon@bestdomain.com,4/14/63,1202,Business,1997,P1,2020-04-07,4.1497
1,1,30,Abdalahe,Aart,10030,Maymana,Afghanistan,abdalahe.aart@bestdomain.com,10.07.40,965,Basic,965,P3,2020-02-21,31.9070
2,2,30,Abdalahe,Aart,10030,Maymana,Afghanistan,abdalahe.aart@bestdomain.com,10.07.40,965,Basic,1760,P4,2020-03-09,31.1756
3,3,48,Abdelaziz,Abaev,10048,Gereshk,Afghanistan,abdelaziz.abaev@bestdomain.com,8/17/85,1189,Business,1984,P1,2020-02-17,25.0801
4,4,55,Abdelfatah,Abaimov,10055,Bāmyān,Afghanistan,abdelfatah.abaimov@bestdomain.com,10.07.40,946,Basic,946,P3,2020-03-11,18.0259
5,5,85,Abdeljebbar,Abalov,10085,Berat,Albania,abdeljebbar.abalov@bestdomain.com,11.01.48,1135,Business,1930,P1,2020-04-07,31.6594
6,6,85,Abdeljebbar,Abalov,10085,Berat,Albania,abdeljebbar.abalov@bestdomain.com,11.01.48,1197,Business,1992,P1,2020-03-19,37.2273
7,7,92,Abdelkarin,Abankin,10092,Ejmiatsin,Armenia,abdelkarin.abankin@bestdomain.com,03.05.67,1451,Business,2246,P1,2020-02-24,21.1675
8,8,112,Abdelmayid,Abashin,10112,Luanda,Angola,abdelmayid.abashin@bestdomain.com,2/24/57,630,Business Premium,630,P3,2020-04-30,0.3188
9,9,114,Abdelmoghit,Abasolo,10114,Cabinda,Angola,abdelmoghit.abasolo@bestdomain.com,8/31/64,1123,Business,1123,P3,2020-01-30,18.0785


In [3]:
df.count()

id                    2554
id_person             2554
name                  1747
surname               2526
zip                   2554
city                  2554
country               2554
email                 2554
birth_date            2554
id_account            2554
account_type          2554
id_transaction        2554
transaction_type      2554
transaction_date      2554
transaction_amount    2554
dtype: int64

In [4]:
df.dtypes

id                      int64
id_person               int64
name                   object
surname                object
zip                     int64
city                   object
country                object
email                  object
birth_date             object
id_account              int64
account_type           object
id_transaction          int64
transaction_type       object
transaction_date       object
transaction_amount    float64
dtype: object

# Step 2: Mapping Data and Connecting the Schema

We are using SQLAlchemy, SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

SQLAlchemy is most famous for its object-relational mapper (ORM), an optional component that provides the data mapper pattern, where classes can be mapped to the database in open ended, multiple ways - allowing the object model and database schema to develop in a cleanly decoupled way from the beginning.

In [5]:
!{sys.executable} -m pip install pip install SQLAlchemy

import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Float, Text

Creating a relationdal datbase name enote2.sqlite. 

In [6]:
engine = create_engine('sqlite:///enote2.sqlite')

# Construct a base class for declarative class definitions.

The new base class will be given a metaclass that produces appropriate Table objects and makes the appropriate mapper() calls based on the information provided declaratively in the class and any subclasses of the class.

bind – An optional Connectable, will be assigned the bind attribute on the MetaData instance.

In [7]:
Base = declarative_base(bind=engine)

In [8]:
class enote(Base):
    __tablename__ = 'enote_data'
    id = Column(Integer, primary_key=True)
    id_person = Column(Integer)
    name = Column(String(15))
    surname = Column(String(15))
    zip = Column(Integer)
    city = Column(String(15))
    country = Column(String(15))
    email = Column(String(15))
    birth_date = Column(String(15))
    id_account = Column(Integer)
    account_type = Column(String(15))
    id_transaction = Column(Integer)
    transaction_type = Column(String(15))
    transaction_date = Column(String(15))
    transaction_amount = Column(Float(6,4))

In [9]:
Base.metadata.create_all()

# Step 3: Migrating Dataframe into Schema

In [10]:
enote_measurements = df.to_dict(orient='records')

In [11]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [12]:
table1_measurement = sqlalchemy.Table('enote_data', metadata, autoload=True)

In [13]:
engine.execute(table1_measurement.delete())

In [14]:
engine.execute(table1_measurement.insert(), enote_measurements)

# Testing the tables that store the migrated data

In [15]:
engine.execute("SELECT * FROM enote_data LIMIT 5").fetchall()

[(0, 13, 'Aaya', 'Añon', 10013, 'Ajman', 'United Arab Emirates', 'aaya.añon@bestdomain.com', '4/14/63', 1202, 'Business', 1997, 'P1', '2020-04-07', 4.1497),
 (1, 30, 'Abdalahe', 'Aart', 10030, 'Maymana', 'Afghanistan', 'abdalahe.aart@bestdomain.com', '10.07.40', 965, 'Basic', 965, 'P3', '2020-02-21', 31.906999999999996),
 (2, 30, 'Abdalahe', 'Aart', 10030, 'Maymana', 'Afghanistan', 'abdalahe.aart@bestdomain.com', '10.07.40', 965, 'Basic', 1760, 'P4', '2020-03-09', 31.1756),
 (3, 48, 'Abdelaziz', 'Abaev', 10048, 'Gereshk', 'Afghanistan', 'abdelaziz.abaev@bestdomain.com', '8/17/85', 1189, 'Business', 1984, 'P1', '2020-02-17', 25.0801),
 (4, 55, 'Abdelfatah', 'Abaimov', 10055, 'Bāmyān', 'Afghanistan', 'abdelfatah.abaimov@bestdomain.com', '10.07.40', 946, 'Basic', 946, 'P3', '2020-03-11', 18.0259)]

In [16]:
engine.execute("SELECT id_person, strftime('%m.%Y', transaction_date) as month, SUM(transaction_amount) as sum_of_transactions FROM enote_data WHERE id_person IN ('1234' , '345') AND transaction_date BETWEEN '2020-02-15' and '2020-07-06' GROUP BY id_person, strftime('%m.%Y', transaction_date) ORDER BY id_person DESC").fetchall()

[(1234, '02.2020', 553.2275000000001),
 (1234, '03.2020', 1223.5367999999994),
 (1234, '04.2020', 802.5819),
 (1234, '05.2020', 400.3059),
 (345, '02.2020', 2644.141200000001),
 (345, '03.2020', 6663.237200000001),
 (345, '04.2020', 3242.386600000001),
 (345, '05.2020', 1712.4352000000003)]

We also got the results like this but we are missing the headers. We can also use sqlite3 and pandas to do
the queries. We will read the enote2.sqlite database and d the query.

In [17]:
import pandas as pd
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("enote2.sqlite")

cur = con.cursor()

In [18]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("enote2.sqlite")
df = pd.read_sql_query("SELECT id_person, strftime('%m.%Y', transaction_date) as month, SUM(transaction_amount) as sum_of_transactions FROM enote_data WHERE id_person IN ('1234' , '345') AND transaction_date BETWEEN '2020-02-15' and '2020-06-06' GROUP BY id_person, strftime('%m.%Y', transaction_date) ORDER BY id_person DESC", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

   id_person    month  sum_of_transactions
0       1234  02.2020             553.2275
1       1234  03.2020            1223.5368
2       1234  04.2020             802.5819
3       1234  05.2020             400.3059
4        345  02.2020            2644.1412


In [19]:
df

,id_person,month,sum_of_transactions
0,1234,02.2020,553.2275
1,1234,03.2020,1223.5368
2,1234,04.2020,802.5819
3,1234,05.2020,400.3059
4,345,02.2020,2644.1412
5,345,03.2020,6663.2372
6,345,04.2020,3242.3866
7,345,05.2020,1712.4352


We migrated the csv files into the relational database and did the query to get the desired results. Now we have a relational datbase in out folder. We can read the relational database from python and execute the queries. We can also modify the relational datbase we created. Let's check the tables in the relational
database enote2.sqlite

In [20]:
from sqlalchemy import create_engine, inspect, func

In [21]:
engine = create_engine('sqlite:///enote2.sqlite')

In [22]:
inspector_gadget = inspect(engine)

Now we can check the tables in our relational datbase called enote2.sqlite

In [23]:
inspector_gadget.get_table_names()

['enote_data']

We can see we have only one table called enote_data. 

In [24]:
print("tablename: enote_data \n")
for piece in inspector_gadget.get_columns(table_name='enote_data'):
    print(piece['name'], piece['type'])

tablename: enote_data 

id INTEGER
id_person INTEGER
name VARCHAR(15)
surname VARCHAR(15)
zip INTEGER
city VARCHAR(15)
country VARCHAR(15)
email VARCHAR(15)
birth_date VARCHAR(15)
id_account INTEGER
account_type VARCHAR(15)
id_transaction INTEGER
transaction_type VARCHAR(15)
transaction_date VARCHAR(15)
transaction_amount FLOAT
